In [1]:
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from scipy.linalg import sqrtm
import numpy as np
import random
from matplotlib import pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Subset, ConcatDataset
from torchvision.datasets import MNIST, CIFAR10
import sys
from tqdm import tqdm
import torch.nn.functional as F
from torch.distributions.multivariate_normal import MultivariateNormal
import copy
import warnings
warnings.filterwarnings("ignore")
import torchvision.models as models
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [2]:
# image0 = np.load("/gfshome/cifar5m-part0.npz")
# image1 = np.load("/gfshome/cifar5m-part1.npz")
# X_tensor = torch.concatenate([torch.tensor(image0['X'], device = device).permute(0, 3, 1, 2), torch.tensor(image1['X'], device = device).permute(0, 3, 1, 2)], dim=0)
# Y_tensor = torch.concatenate([torch.tensor(image0['Y'], device = device), torch.tensor(image1['Y'], device = device)], dim=0)
# X_tensor = torch.tensor(image1['X'], device = device).permute(0, 3, 1, 2)
# Y_tensor = image1['Y']
# mask_0 = (Y_tensor == 0)
# mask_1 = (Y_tensor == 1)
# X_train_tensor_0 = X_tensor[mask_0][:100000,:,:,:]/256
# X_train_tensor_1 = X_tensor[mask_1][:100000,:,:,:]/256
# X_test_tensor_0 = X_tensor[mask_0][100000:150000,:,:,:]/256
# X_test_tensor_1 = X_tensor[mask_1][:50000,:,:,:]/256
# X_train_tensor_0 = transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))(X_train_tensor_0)
# X_train_tensor_1 = transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))(X_train_tensor_1)
# X_test_tensor_0 = transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))(X_test_tensor_0)
# X_test_tensor_1 = transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))(X_test_tensor_1)

In [3]:
# class LogisticRegression(torch.nn.Module):
#     def __init__(self, input_dim, fit_intercept=True, C=1.0, max_iter=100):
#         super(LogisticRegression, self).__init__()
#         self.fit_intercept = fit_intercept
#         self.C = C
#         self.max_iter = max_iter
        
#         if self.fit_intercept:
#             self.linear = nn.Linear(input_dim, 1)
#         else:
#             self.linear = nn.Linear(input_dim, 1, bias=False)
        
#     def forward(self, x):
#         return torch.sigmoid(self.linear(x))
    
#     def fit(self, X, y):
#         X = torch.tensor(X, dtype=torch.float32)
#         y = torch.tensor(y, dtype=torch.float32).view(-1, 1)
        
#         criterion = nn.BCELoss()
#         optimizer = optim.SGD(self.parameters(), lr=1e-3)
        
#         for epoch in range(self.max_iter):
#             optimizer.zero_grad()
#             outputs = self.forward(X)
#             loss = criterion(outputs, y)
            
#             l2_lambda = 1.0 / self.C
#             l2_reg = sum(param.pow(2.0).sum() for param in self.parameters())
#             loss = loss + l2_lambda * l2_reg
            
#             loss.backward()
#             optimizer.step()
        
#         self.coef_ = self.linear.weight.detach().cpu().numpy()
#         if self.fit_intercept:
#             self.intercept_ = self.linear.bias.detach().cpu().numpy()
#         else:
#             self.intercept_ = None

#     def predict_proba(self, X):
#         X = torch.tensor(X, dtype=torch.float32)
#         return self.forward(X).detach().cpu().numpy()

#     def predict(self, X):
#         proba = self.predict_proba(X)
#         return (proba >= 0.5).astype(int)
    
#     def score(self, X, y):
#         X = torch.tensor(X, dtype=torch.float32)
#         y = torch.tensor(y, dtype=torch.float32).view(-1, 1)
#         y_pred = self.predict(X)
#         accuracy = (y_pred == y.cpu().numpy()).mean()
#         return accuracy


In [4]:
N_large = 400
N_small = 100
mu = 1.0
var = 2.0
D = 3
T = 3
test_size_L, test_size_H, test_size_step = 30, 50, 10
train_size = 800
candidate_L, candidate_H, candidate_step = 200, 600, 200
top_candidate = [3, 9, 27]
penalty = 5
eps = 0.01
width = 0.15
gap = 0.05
MCsample = 500
sigma = 1e-1
p = 10000

In [5]:
# resnet18 = models.resnet18(pretrained=True)
# resnet18.fc = torch.nn.Identity()
# resnet18.to(device).eval()
# print(resnet18)

In [6]:
# with torch.no_grad():
#     X_train_0_embedding = resnet18(X_train_tensor_0)
#     X_train_1_embedding = resnet18(X_train_tensor_1)
#     X_test_0_embedding = resnet18(X_test_tensor_0)
    # X_test_1_embedding = resnet18(X_test_tensor_1)
#     torch.save(X_train_0_embedding, "/gfshome/train_0.pt")
#     torch.save(X_train_1_embedding, "/gfshome/train_1.pt")
#     torch.save(X_test_0_embedding, "/gfshome/test_0.pt")
    # torch.save(X_test_1_embedding, "/gfshome/test_1.pt")

In [7]:
X_train_0_embedding = torch.load("/gfshome/train_0.pt")
X_train_1_embedding = torch.load("/gfshome/train_1.pt")
X_train_0_embedding = torch.concatenate([X_train_0_embedding, torch.ones(X_train_0_embedding.size()[0],1).to(device)], dim=1)
X_train_1_embedding = torch.concatenate([X_train_1_embedding, torch.ones(X_train_1_embedding.size()[0],1).to(device)], dim=1)
X_test_0_embedding = torch.load("/gfshome/test_0.pt")
X_test_1_embedding = torch.load("/gfshome/test_1.pt")
P_x = torch.concatenate([X_test_0_embedding[:p], X_test_1_embedding[:p]])
P_x = torch.concatenate([P_x, torch.ones(P_x.size()[0],1).to(device)], dim=1)
P_y = torch.concatenate([torch.zeros(p), torch.ones(p)]).to(device)
# allocated_memory = torch.cuda.memory_allocated()
# print(f"Already allocated: {allocated_memory / 1024 ** 2} MB")
# print(X_test_0_embedding.size())
input_dim = P_x.size()[1]

In [8]:
class dataset:
    def __init__(self, X, y, score, post_loss, base_loss, smooth, post_acc, base_acc, noise_ratio, label_ratio):
        self.X = X
        self.y = y
        self.score = score
        self.post_loss = post_loss
        self.base_loss = base_loss
        self.smooth = smooth
        self.post_acc = post_acc
        self.base_acc = base_acc
        self.noise_ratio = noise_ratio
        self.label_ratio = label_ratio

class result:
    def __init__(self, score_loss, score_acc, post_loss_loss, post_loss_acc, base_loss_loss, base_loss_acc, smooth_loss, smooth_acc, post_acc_loss, post_acc_acc, base_acc_loss, base_acc_acc):
        self.score_loss = score_loss
        self.score_acc = score_acc
        self.post_loss_loss = post_loss_loss
        self.post_loss_acc = post_loss_acc
        self.base_loss_loss = base_loss_loss
        self.base_loss_acc = base_loss_acc
        self.smooth_loss = smooth_loss
        self.smooth_acc = smooth_acc
        self.post_acc_loss = post_acc_loss
        self.post_acc_acc = post_acc_acc
        self.base_acc_loss = base_acc_loss
        self.base_acc_acc = base_acc_acc

    def add(self, another):        
        self.score_loss += another.score_loss
        self.score_acc += another.score_acc
        self.post_loss_loss += another.post_loss_loss
        self.post_loss_acc += another.post_loss_acc
        self.base_loss_loss += another.base_loss_loss
        self.base_loss_acc += another.base_loss_acc
        self.smooth_loss += another.smooth_loss
        self.smooth_acc += another.smooth_acc
        self.post_acc_loss += another.post_acc_loss
        self.post_acc_acc += another.post_acc_acc
        self.base_acc_loss += another.base_acc_loss
        self.base_acc_acc += another.base_acc_acc

    def divide(self, num):        
        self.score_loss = self.score_loss / num
        self.score_acc = self.score_acc / num
        self.post_loss_loss = self.post_loss_loss / num
        self.post_loss_acc = self.post_loss_acc / num
        self.base_loss_loss = self.base_loss_loss / num
        self.base_loss_acc = self.base_loss_acc / num
        self.smooth_loss = self.smooth_loss / num
        self.smooth_acc = self.smooth_acc / num
        self.post_acc_loss = self.post_acc_loss / num
        self.post_acc_acc = self.post_acc_acc / num
        self.base_acc_loss = self.base_acc_loss / num
        self.base_acc_acc = self.base_acc_acc / num

    def getacc(self):
        return [self.score_acc, self.post_loss_acc, self.base_loss_acc, self.smooth_acc, self.post_acc_acc, self.base_acc_acc]
    
    def getloss(self):
        return [self.score_loss, self.post_loss_loss, self.base_loss_loss, self.smooth_loss, self.post_acc_loss, self.base_acc_loss]

#    def self_print(self, num):
#        print("score: ", self.score_loss/num, self.score_acc/num, "\nlog loss:", self.err_loss/num, self.err_acc/num, "\naccuracy:", self.acc_loss/num, self.acc_acc/num)


class summary:
    def __init__(self, percentage_smooth_post, percentage_score_post, percentage_post_base_loss, percentage_post_base_acc, total_result):
        self.percentage_smooth_post = percentage_smooth_post
        self.percentage_score_post = percentage_score_post
        self.percentage_post_base_loss = percentage_post_base_loss
        self.percentage_post_base_acc = percentage_post_base_acc
        self.total_result = total_result
        
    def add(self, another):
        self.percentage_smooth_post = np.add(self.percentage_smooth_post, another.percentage_smooth_post)
        self.percentage_score_post = np.add(self.percentage_score_post, another.percentage_score_post)
        self.percentage_post_base_loss = np.add(self.percentage_post_base_loss, another.percentage_post_base_loss)
        self.percentage_post_base_acc = np.add(self.percentage_post_base_acc, another.percentage_post_base_acc)
        self.total_result.add(another.total_result)
        
    def divide(self, num):
        self.percentage_smooth_post = self.percentage_smooth_post / num
        self.percentage_score_post = self.percentage_score_post / num 
        self.percentage_post_base_loss = self.percentage_post_base_loss / num 
        self.percentage_post_base_acc = self.percentage_post_base_acc / num
        self.total_result.divide(num)

    def getacc(self):
        return self.total_result.getacc()
    
    def getloss(self):
        return self.total_result.getloss()

#    def self_print_percentage(self, num):
#        print(self.percentage_loss_err/num, self.percentage_loss_acc/num, self.percentage_acc_err/num, self.percentage_acc_acc/num)

#    def self_print_result(self, num):
#        self.total_result.self_print(num)

In [9]:
def sigmoid(z):
    return 1/(1 + torch.exp(-z))

def generate_data_cifar10(num, dataset, label):
    indices = torch.randperm(dataset.size(0))[:num]
    X = dataset[indices]
    y = torch.ones(num) * label
    y = torch.tensor(y, dtype=torch.int)
    y = y.to(device)
    return X, y

def generate_dataset_cifar10(N, noise):
    N_0 = random.randint(1,N-1)
    X0, y0 = generate_data_cifar10(N_0, X_train_0_embedding, 0)
    X1, y1 = generate_data_cifar10(N-N_0, X_train_1_embedding, 1)

    X = torch.concatenate((X0, X1), axis = 0)
    y = torch.concatenate((y0, y1), axis = 0)
    for i in range(N):
        flip = torch.bernoulli(torch.tensor(noise)).item()
        if flip == 1:
            y[i] = 1 - y[i]
    return N_0/N, X, y

def generate_train_cifar10(train_size, train_number):
    train_dataset = []
    for i in range(train_number):
        noise_level = random.randint(1, 30)
        # noise_level = 0
        not_found = True
        while not_found:
            label_ratio, train_X, train_y = generate_dataset_cifar10(train_size, eps * noise_level)
            if torch.sum(train_y)>0 and torch.sum(train_y)<train_size:
                not_found = False
        train_dataset.append(dataset(train_X, train_y, 0, 0, 0, 0, 0, 0, noise_level*eps, label_ratio))
    return train_dataset

def subsample(X, y, size):
    perm = torch.randperm(len(y))
    sample_X = X[perm[:size]]
    sample_y = y[perm[:size]]
    return sample_X, sample_y

def compute_hessian(mu, X):
    m = X.size()[1]
    res = torch.eye(m)
    res = res.to(device)
    for i in range(X.size()[0]):
        xi = X[i:i+1, :]
        sigm = sigmoid(torch.dot(xi[0], mu[0]))
        res += torch.matmul(xi.t(), xi) * (sigm * (1 - sigm)).item()
    return res

def compute_score(mu0, Q0, lg0, mu1, Q1, lg1, mu2, Q2, lg2):
    Q = Q1 + Q2 - Q0
    Q_inv = torch.inverse(Q)
    mu = torch.matmul(Q_inv, torch.matmul(Q1, mu1) + torch.matmul(Q2, mu2) - torch.matmul(Q0, mu0))

    # Q_numpy = Q.detach().cpu().numpy()
    # eigenvalue, _ = np.linalg.eigh(Q_numpy)
    # eigenvalue = torch.tensor(eigenvalue)
    # lg12 = torch.sum(torch.log(eigenvalue))
    eigenvalue, _ = torch.linalg.eigh(Q)
    lg12 = torch.sum(torch.log(eigenvalue))

    lg = lg1+lg2-lg12-lg0

    sqr = torch.matmul(mu.T, torch.matmul(Q, mu)) - torch.matmul(mu1.T, torch.matmul(Q1, mu1)) - torch.matmul(mu2.T, torch.matmul(Q2, mu2)) + torch.matmul(mu0.T, torch.matmul(Q0, mu0))

    score = 0.5 * (lg + sqr)
    # print(lg.item(), sqr.item())
    
    return score.item()

def compute_data_score_err(mu_test, Q_test, test_X, test_y, train_X, train_y, lg2):
    test_N = test_y.size()[0]
    M = test_X.size()[1]

    mu0 = torch.zeros((1, M))
    mu0 = mu0.to(device)
    Q0 = torch.eye(M)/penalty
    Q0 = Q0.to(device)
    lg0 = -M * torch.log(torch.tensor(penalty))
    
    train = LogisticRegression(fit_intercept = False, C = penalty, max_iter=5000).fit(train_X.cpu(), train_y.cpu())
    mu_train = torch.tensor(train.coef_, dtype=torch.float32, device=device)
    mu_train_numpy = mu_train.detach().squeeze().cpu().numpy()

    Q_train = compute_hessian(mu_train, train_X)
    Q_train_inverse = torch.inverse(Q_train)
    Q_numpy = Q_train_inverse.detach().cpu().numpy()

    # Q_train_numpy = Q_train.detach().cpu().numpy()
    # eigenvalue, _ = np.linalg.eigh(Q_train_numpy)
    # eigenvalue = torch.tensor(eigenvalue)
    # lg1 = torch.sum(torch.log(eigenvalue))
    eigenvalue, _ = torch.linalg.eigh(Q_train)
    lg1 = torch.sum(torch.log(eigenvalue))

    score = compute_score(mu0.t(), Q0, lg0, mu_train.t(), Q_train, lg1, mu_test.t(), Q_test, lg2)
    # print(score)

    # log_loss = torch.zeros(test_N)
    # log_loss = log_loss.to(device)
    # pred = torch.tensor(train.predict_proba(test_X.cpu()), dtype=torch.float32, device=device)
    
#     for j in range(test_N):
#         a = test_y[j]
#         b = pred[j,1]
# #        print(a,b)
#         log_loss[j] = a * torch.log(b) + (1 - a) * torch.log(1 - b)
        
#     test_err = torch.sum(log_loss)/test_N
#     test_acc = train.score(test_X.cpu(), test_y.cpu())

    # z = 2 * test_y - 1
    # z = z.view(-1,1)
    test_y = test_y.float()
    criterion = nn.BCELoss()

    base_predictive = sigmoid(torch.matmul(test_X, mu_train.t())).squeeze()
    base_predictions = (base_predictive >= 0.5).float()
    base_loss = criterion(base_predictive, test_y)
    base_acc = (base_predictions == test_y).float().mean()

    # mvn = torch.distributions.MultivariateNormal(mu_train, Q_train_inverse)
    # theta_samples = mvn.sample((MCsample, ))
    # print(mu_train_numpy.shape)
    theta_samples = np.random.multivariate_normal(mu_train_numpy, Q_numpy, MCsample, 'warn')
    theta_samples = torch.tensor(theta_samples).to(device)
    theta_samples = theta_samples.float()

    post_predictive = torch.mean(sigmoid(torch.matmul(test_X, theta_samples.t())), dim=1)
    post_predictions = (post_predictive >= 0.5).float()
    post_loss = criterion(post_predictive, test_y)
    post_acc = (post_predictions == test_y).float().mean()

    Q_numpy_smooth = sigma * np.eye(M)
    theta_samples_smooth = np.random.multivariate_normal(mu_train_numpy, Q_numpy_smooth, MCsample, 'warn')
    theta_samples_smooth = torch.tensor(theta_samples_smooth).to(device)
    theta_samples_smooth = theta_samples_smooth.float()

    smooth_predictive = torch.mean(sigmoid(torch.matmul(test_X, theta_samples_smooth.t())), dim=1)
    # smooth_predictions = (smooth_predictive >= 0.5).float()
    smooth_loss = criterion(smooth_predictive, test_y)
    # smooth_acc = (smooth_predictions == test_y).float().mean()
    
    return score, base_loss.item(), base_acc, post_loss.item(), post_acc, smooth_loss.item()

def get_err_score(train_data, test_X, test_y, train_number):
    test = LogisticRegression(fit_intercept = False, C = penalty, max_iter=5000).fit(test_X.cpu(), test_y.cpu())
    mu_test = torch.tensor(test.coef_, dtype=torch.float32, device=device)
    Q_test = compute_hessian(mu_test, test_X)
    # Q_test_numpy = Q_test.detach().cpu().numpy()
    # eigenvalue, _ = np.linalg.eigh(Q_test_numpy)
    # eigenvalue = torch.tensor(eigenvalue)
    # lg2 = torch.sum(torch.log(eigenvalue))
    eigenvalue, _ = torch.linalg.eigh(Q_test)
    lg2 = torch.sum(torch.log(eigenvalue))

    for i in range(train_number):
        train_data[i].score, train_data[i].base_loss, train_data[i].base_acc, train_data[i].post_loss, train_data[i].post_acc, train_data[i].smooth = compute_data_score_err(mu_test, Q_test, test_X, test_y, train_data[i].X, train_data[i].y, lg2)

def compute_data_err_acc(test_X, test_y, train_X, train_y):
    test_N = test_y.size()[0]
    train = LogisticRegression(fit_intercept = False, C = penalty, max_iter=5000).fit(train_X.cpu(), train_y.cpu())
    mu_train = torch.tensor(train.coef_, dtype=torch.float32, device=device)
    mu_train_numpy = mu_train.detach().squeeze().cpu().numpy()
    Q_train = compute_hessian(mu_train, train_X)
    Q_train_inverse = torch.inverse(Q_train)
    Q_numpy = Q_train_inverse.detach().cpu().numpy()

#     log_loss = torch.zeros(test_N)
#     log_loss = log_loss.to(device)
#     pred = torch.tensor(train.predict_proba(test_X.cpu()), dtype=torch.float32, device=device)

#     for j in range(test_N):
#         a = test_y[j]
#         b = pred[j,1]
# #        print(a,b)
#         log_loss[j] = a * torch.log(b) + (1 - a) * torch.log(1 - b)
        
#     test_err = torch.sum(log_loss)/test_N
#     test_acc = train.score(test_X.cpu(), test_y.cpu())
    test_y = test_y.float()
    criterion = nn.BCELoss()

    base_predictive = sigmoid(torch.matmul(test_X, mu_train.t())).squeeze()
    base_predictions = (base_predictive >= 0.5).float()
    base_loss = criterion(base_predictive, test_y)
    base_acc = (base_predictions == test_y).float().mean()

    theta_samples = np.random.multivariate_normal(mu_train_numpy, Q_numpy, MCsample, 'warn')
    theta_samples = torch.tensor(theta_samples).to(device)
    theta_samples = theta_samples.float()

    # z = 2 * test_y - 1
    # z = z.view(-1,1)
    # print(z.view(-1, 1) * torch.matmul(test_X, theta_samples.t()))
    # post_predict = (1 / MCsample) * torch.sum(sigmoid(torch.matmul(test_X, theta_samples.t())), dim=1)

    post_predictive = torch.mean(sigmoid(torch.matmul(test_X, theta_samples.t())), dim=1)
    post_predictions = (post_predictive >= 0.5).float()
    post_loss = criterion(post_predictive, test_y)
    post_acc = (post_predictions == test_y).float().mean()


    # loss = criterion(post_predict, test_y.float())
    # predicted = (post_predict > 0.5).float()
    # accuracy = (predicted == test_y.float()).float().mean()

    # print(post_predict)
    # sys.exit()
    return base_loss.item(), base_acc, post_loss.item(), post_acc

In [10]:
def generate_plot(x_label, a_L, a_H, label1, label2, save_name):
#    print(width, gap)
    x = np.array(range(len(x_label))) * (width + gap)
#    print(e_L, e_H)
    acc_L = np.ones(len(x_label))*100 - np.array(a_H) 
    acc_H = np.ones(len(x_label))*100 - np.array(a_L)
    plt.rcParams['figure.figsize'] = [(width + gap) * (len(x_label) + 1), 4]
    plt.bar(x, acc_L, color='sandybrown', width = width, label = label1)
    plt.bar(x, np.ones(len(x_label))*100 - acc_H, bottom= acc_H, color='cornflowerblue', width = width, label = label2)
    plt.bar(x, acc_H - acc_L, bottom= acc_L, color='lightgray', width = width, label = 'Ties')
    plt.xticks(x, x_label, rotation='vertical')
    plt.tight_layout()
    plt.legend(loc=4)
    plt.savefig("figure/"+save_name+".pdf", format="pdf", dpi=200) #add legend
    plt.show()

In [11]:
from pyinstrument import Profiler

# profiler = Profiler()
# profiler.start()
x_label_all = []

single_smooth_post_L_all = []
single_smooth_post_H_all = []
single_score_post_L_all = []
single_score_post_H_all = []
single_post_base_loss_L_all = []
single_post_base_loss_H_all = []
single_post_base_acc_L_all = []
single_post_base_acc_H_all = []

smooth_post_L_all = []
smooth_post_H_all = []
score_post_L_all = []
score_post_H_all = []
post_base_loss_L_all = []
post_base_loss_H_all = []
post_base_acc_L_all = []
post_base_acc_H_all = []

single_acc_all_all = []
acc_all_all = []
single_loss_all_all = []
loss_all_all = []

for test_size in range(test_size_L, test_size_H+1, test_size_step):
    x_label = []

    single_smooth_post_L = []
    single_smooth_post_H = []
    single_score_post_L = []
    single_score_post_H = []
    single_post_base_loss_L = []
    single_post_base_loss_H = []
    single_post_base_acc_L = []
    single_post_base_acc_H= []

    smooth_post_L = []
    smooth_post_H = []
    score_post_L = []
    score_post_H = []
    post_base_loss_L = []
    post_base_loss_H = []
    post_base_acc_L = []
    post_base_acc_H = []

    single_acc_all = []
    acc_all = []
    single_loss_all = []
    loss_all = []

    for num_candidate in range(candidate_L, candidate_H+1, candidate_step):
        single_total_summary = []
        total_summary = []
        for i in range(len(top_candidate)):
            single_total_summary.append(summary([0,0], [0,0], [0,0], [0,0], result(0,0,0,0,0,0,0,0,0,0,0,0)))
            total_summary.append(summary([0,0], [0,0], [0,0], [0,0], result(0,0,0,0,0,0,0,0,0,0,0,0)))
        for d in tqdm(range(D)):
            test_X = P_x
            test_y = P_y
            test = LogisticRegression(fit_intercept = False, C = penalty, max_iter=5000).fit(test_X.cpu(), test_y.cpu())
            acc = test.score(test_X.cpu(), test_y.cpu())
            print(acc)
            sys.exit()

            train_data = generate_train_cifar10(train_size, num_candidate)
            single_total_results = []
            total_results = []
            single_results = []
            results = []
            for i in range(len(top_candidate)):
                single_total_results.append(result(0,0,0,0,0,0,0,0,0,0,0,0))
                total_results.append(result(0,0,0,0,0,0,0,0,0,0,0,0))
                single_results.append([])
                results.append([])
            for t in range(T):
                not_found = True
                while not_found:
                    sample_test_X, sample_test_y = subsample(test_X, test_y, test_size)
                    if torch.sum(sample_test_y)>0 and torch.sum(sample_test_y)<test_size:
                        not_found = False
                
                get_err_score(train_data, sample_test_X, sample_test_y, num_candidate)
                sorted_score = sorted(train_data, key=lambda x: x.score, reverse = True)
                sorted_post_loss = sorted(train_data, key=lambda x: x.post_loss, reverse = False)
                sorted_base_loss = sorted(train_data, key=lambda x: x.base_loss, reverse = False)
                sorted_smooth = sorted(train_data, key=lambda x: x.smooth, reverse = False)
                sorted_post_acc = sorted(train_data, key=lambda x: x.post_acc, reverse = True)
                sorted_base_acc = sorted(train_data, key=lambda x: x.base_acc, reverse = True)

                score_X = sorted_score[0].X
                score_y = sorted_score[0].y
                post_loss_X = sorted_post_loss[0].X
                post_loss_y = sorted_post_loss[0].y
                base_loss_X = sorted_base_loss[0].X
                base_loss_y = sorted_base_loss[0].y
                smooth_X = sorted_smooth[0].X
                smooth_y = sorted_smooth[0].y
                post_acc_X = sorted_post_acc[0].X
                post_acc_y = sorted_post_acc[0].y
                base_acc_X = sorted_base_acc[0].X
                base_acc_y = sorted_base_acc[0].y

                for j in range(1, top_candidate[-1]):
                    score_X = torch.concatenate((score_X, sorted_score[j].X), axis = 0)
                    score_y = torch.concatenate((score_y, sorted_score[j].y), axis = 0)
                    post_loss_X = torch.concatenate((post_loss_X, sorted_post_loss[j].X), axis = 0)
                    post_loss_y = torch.concatenate((post_loss_y, sorted_post_loss[j].y), axis = 0)
                    base_loss_X = torch.concatenate((base_loss_X, sorted_base_loss[j].X), axis = 0)
                    base_loss_y = torch.concatenate((base_loss_y, sorted_base_loss[j].y), axis = 0)
                    smooth_X = torch.concatenate((smooth_X, sorted_smooth[j].X), axis = 0)
                    smooth_y = torch.concatenate((smooth_y, sorted_smooth[j].y), axis = 0)
                    post_acc_X = torch.concatenate((post_acc_X, sorted_post_acc[j].X), axis = 0)
                    post_acc_y = torch.concatenate((post_acc_y, sorted_post_acc[j].y), axis = 0)
                    base_acc_X = torch.concatenate((base_acc_X, sorted_base_acc[j].X), axis = 0)
                    base_acc_y = torch.concatenate((base_acc_y, sorted_base_acc[j].y), axis = 0)

                for i in range(len(top_candidate)):
                    single_score_loss, single_score_acc, score_loss, score_acc = compute_data_err_acc(test_X, test_y, score_X[:int(top_candidate[i]*train_size)], score_y[:int(top_candidate[i]*train_size)])
                    single_post_loss_loss, single_post_loss_acc, post_loss_loss, post_loss_acc = compute_data_err_acc(test_X, test_y, post_loss_X[:int(top_candidate[i]*train_size)], post_loss_y[:int(top_candidate[i]*train_size)])
                    single_base_loss_loss, single_base_loss_acc, base_loss_loss, base_loss_acc = compute_data_err_acc(test_X, test_y, base_loss_X[:int(top_candidate[i]*train_size)], base_loss_y[:int(top_candidate[i]*train_size)])
                    single_smooth_loss, single_smooth_acc, smooth_loss, smooth_acc = compute_data_err_acc(test_X, test_y, smooth_X[:int(top_candidate[i]*train_size)], smooth_y[:int(top_candidate[i]*train_size)])
                    single_post_acc_loss, single_post_acc_acc, post_acc_loss, post_acc_acc = compute_data_err_acc(test_X, test_y, post_acc_X[:int(top_candidate[i]*train_size)], post_acc_y[:int(top_candidate[i]*train_size)])
                    single_base_acc_loss, single_base_acc_acc, base_acc_loss, base_acc_acc = compute_data_err_acc(test_X, test_y, base_acc_X[:int(top_candidate[i]*train_size)], base_acc_y[:int(top_candidate[i]*train_size)])

                    single_resi = result(single_score_loss, single_score_acc, single_post_loss_loss, single_post_loss_acc, single_base_loss_loss, single_base_loss_acc, single_smooth_loss, single_smooth_acc, single_post_acc_loss, single_post_acc_acc, single_base_acc_loss, single_base_acc_acc)
                    resi = result(score_loss, score_acc, post_loss_loss, post_loss_acc, base_loss_loss, base_loss_acc, smooth_loss, smooth_acc, post_acc_loss, post_acc_acc, base_acc_loss, base_acc_acc)
                    single_results[i].append(single_resi)
                    results[i].append(resi)
                    single_total_results[i].add(single_resi)
                    total_results[i].add(resi)

                    # print(single_resi.getacc(), single_resi.getloss(), resi.getacc(), resi.getloss())
                    # sys.exit()
            for i in range(len(top_candidate)):
                single_total_results[i].divide(T)
                total_results[i].divide(T)
                single_summ = summary([0, 0], [0, 0], [0, 0], [0, 0], single_total_results[i])
                summ = summary([0, 0], [0, 0], [0, 0], [0, 0], total_results[i])

                for t in range(T):
                    if single_results[i][t].smooth_acc >= single_results[i][t].post_loss_acc:
                        single_summ.percentage_smooth_post[1] += 1/T*100
                    if single_results[i][t].smooth_acc > single_results[i][t].post_loss_acc:
                        single_summ.percentage_smooth_post[0] += 1/T*100
                    if single_results[i][t].score_acc >= single_results[i][t].post_loss_acc:
                        single_summ.percentage_score_post[1] += 1/T*100
                    if single_results[i][t].score_acc > single_results[i][t].post_loss_acc:
                        single_summ.percentage_score_post[0] += 1/T*100
                    if single_results[i][t].post_loss_acc >= single_results[i][t].base_loss_acc:
                        single_summ.percentage_post_base_loss[1] += 1/T*100
                    if single_results[i][t].post_loss_acc > single_results[i][t].base_loss_acc:
                        single_summ.percentage_post_base_loss[0] += 1/T*100
                    if single_results[i][t].post_acc_acc >= single_results[i][t].base_acc_acc:
                        single_summ.percentage_post_base_acc[1] += 1/T*100
                    if single_results[i][t].post_acc_acc > single_results[i][t].base_acc_acc:
                        single_summ.percentage_post_base_acc[0] += 1/T*100

                    if results[i][t].smooth_acc >= results[i][t].post_loss_acc:
                        summ.percentage_smooth_post[1] += 1/T*100
                    if results[i][t].smooth_acc > results[i][t].post_loss_acc:
                        summ.percentage_smooth_post[0] += 1/T*100
                    if results[i][t].score_acc >= results[i][t].post_loss_acc:
                        summ.percentage_score_post[1] += 1/T*100
                    if results[i][t].score_acc > results[i][t].post_loss_acc:
                        summ.percentage_score_post[0] += 1/T*100
                    if results[i][t].post_loss_acc >= results[i][t].base_loss_acc:
                        summ.percentage_post_base_loss[1] += 1/T*100
                    if results[i][t].post_loss_acc > results[i][t].base_loss_acc:
                        summ.percentage_post_base_loss[0] += 1/T*100
                    if results[i][t].post_acc_acc >= results[i][t].base_acc_acc:
                        summ.percentage_post_base_acc[1] += 1/T*100
                    if results[i][t].post_acc_acc > results[i][t].base_acc_acc:
                        summ.percentage_post_base_acc[0] += 1/T*100
                single_total_summary[i].add(single_summ)
                total_summary[i].add(summ)
        for i in range(len(top_candidate)):
            single_total_summary[i].divide(D)
            total_summary[i].divide(D)
            label = str(top_candidate[i]) + ", " + str(test_size) + ", " + str(num_candidate)
            x_label.append(label)
            single_acc = single_total_summary[i].getacc()
            acc = total_summary[i].getacc()
            single_loss = single_total_summary[i].getloss()
            loss = total_summary[i].getloss()
            print("$"+label+"$ & " + "{:.4f}".format(single_acc[0]) + " & " + "{:.4f}".format(single_acc[1]) + " & " + "{:.4f}".format(single_acc[2]) + " & " + "{:.4f}".format(single_acc[3]) + " & " + "{:.4f}".format(single_acc[4]) + " & " + "{:.4f}".format(single_acc[5]) +" \\\\ " )
            print("$"+label+"$ & " + "{:.4f}".format(acc[0]) + " & " + "{:.4f}".format(acc[1]) + " & " + "{:.4f}".format(acc[2]) + " & " + "{:.4f}".format(acc[3]) + " & " + "{:.4f}".format(acc[4]) + " & " + "{:.4f}".format(acc[5]) +" \\\\ " )
            print("$"+label+"$ & " + "{:.4f}".format(single_acc[0]-acc[0]) + " & " + "{:.4f}".format(single_acc[1]-acc[1]) + " & " + "{:.4f}".format(single_acc[2]-acc[2]) + " & " + "{:.4f}".format(single_acc[3]-acc[3]) + " & " + "{:.4f}".format(single_acc[4]-acc[4]) + " & " + "{:.4f}".format(single_acc[5]-acc[5]) + " & " )
            print("$"+label+"$ & " + "{:.4f}".format(single_loss[0]) + " & " + "{:.4f}".format(single_loss[1]) + " & " + "{:.4f}".format(single_loss[2]) + " & " + "{:.4f}".format(single_loss[3]) + " & " + "{:.4f}".format(single_loss[4]) + " & " + "{:.4f}".format(single_loss[5]) +" \\\\ " )
            print("$"+label+"$ & " + "{:.4f}".format(loss[0]) + " & " + "{:.4f}".format(loss[1]) + " & " + "{:.4f}".format(loss[2]) + " & " + "{:.4f}".format(loss[3]) + " & " + "{:.4f}".format(loss[4]) + " & " + "{:.4f}".format(loss[5]) +" \\\\ " )
            print("$"+label+"$ & " + "{:.4f}".format(single_loss[0]-loss[0]) + " & " + "{:.4f}".format(single_loss[1]-loss[1]) + " & " + "{:.4f}".format(single_loss[2]-loss[2]) + " & " + "{:.4f}".format(single_loss[3]-loss[3]) + " & " + "{:.4f}".format(single_loss[4]-loss[4]) + " & " + "{:.4f}".format(single_loss[5]-loss[5]) + " & " )
            single_smooth_post_L.append(single_total_summary[i].percentage_smooth_post[0])
            single_smooth_post_H.append(single_total_summary[i].percentage_smooth_post[1])
            single_score_post_L.append(single_total_summary[i].percentage_score_post[0])
            single_score_post_H.append(single_total_summary[i].percentage_score_post[1])
            single_post_base_loss_L.append(single_total_summary[i].percentage_post_base_loss[0])
            single_post_base_loss_H.append(single_total_summary[i].percentage_post_base_loss[1])
            single_post_base_acc_L.append(single_total_summary[i].percentage_post_base_acc[0])
            single_post_base_acc_H.append(single_total_summary[i].percentage_post_base_acc[1])

            smooth_post_L.append(total_summary[i].percentage_smooth_post[0])
            smooth_post_H.append(total_summary[i].percentage_smooth_post[1])
            score_post_L.append(total_summary[i].percentage_score_post[0])
            score_post_H.append(total_summary[i].percentage_score_post[1])
            post_base_loss_L.append(total_summary[i].percentage_post_base_loss[0])
            post_base_loss_H.append(total_summary[i].percentage_post_base_loss[1])
            post_base_acc_L.append(total_summary[i].percentage_post_base_acc[0])
            post_base_acc_H.append(total_summary[i].percentage_post_base_acc[1])

            single_acc_all.append(single_acc)
            acc_all.append(acc)
            single_loss_all.append(single_loss)
            loss_all.append(loss)
        profiler.stop()

        profiler.print()
        sys.exit()
    
    x_label_all += x_label
    single_smooth_post_L_all += single_smooth_post_L
    single_smooth_post_H_all += single_smooth_post_H
    single_score_post_L_all += single_score_post_L
    single_score_post_H_all += single_score_post_H
    single_post_base_loss_L_all += single_post_base_loss_L
    single_post_base_loss_H_all += single_post_base_loss_H
    single_post_base_acc_L_all += single_post_base_acc_L
    single_post_base_acc_H_all += single_post_base_acc_H

    smooth_post_L_all += smooth_post_L
    smooth_post_H_all += smooth_post_H
    score_post_L_all += score_post_L
    score_post_H_all += score_post_H
    post_base_loss_L_all += post_base_loss_L
    post_base_loss_H_all += post_base_loss_H
    post_base_acc_L_all += post_base_acc_L
    post_base_acc_H_all += post_base_acc_H

    single_acc_all_all += single_acc_all
    acc_all_all += acc_all
    single_loss_all_all += single_loss_all
    loss_all_all += loss_all

    generate_plot(x_label, single_smooth_post_L, single_smooth_post_H, 'Posterior Predictive', 'Small Smooth', 'single_smooth_post')
    generate_plot(x_label, single_score_post_L, single_score_post_H, 'Posterior Predictive', 'PMI', 'single_score_post')
    generate_plot(x_label, single_post_base_loss_L, single_post_base_loss_H, 'Cross Entropy', 'Average Cross Entropy', 'single_post_base_loss')
    generate_plot(x_label, single_post_base_acc_L, single_post_base_acc_H, 'Accuracy', 'Average Accuracy', 'single_post_base_acc')
    generate_plot(x_label, smooth_post_L, smooth_post_H, 'Posterior Predictive', 'Small Smooth', 'smooth_post')
    generate_plot(x_label, score_post_L, score_post_H, 'Posterior Predictive', 'PMI', 'score_post')
    generate_plot(x_label, post_base_loss_L, post_base_loss_H, 'Cross Entropy', 'Average Cross Entropy', 'post_base_loss')
    generate_plot(x_label, post_base_acc_L, post_base_acc_H, 'Accuracy', 'Average Accuracy', 'post_base_acc')

# profiler.stop()

# profiler.print()


  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [1:16:02<00:00, 1520.89s/it]


$3, 30, 200$ & 0.9045 & 0.8741 & 0.8692 & 0.8841 & 0.8637 & 0.8684 \\ 
$3, 30, 200$ & 0.9043 & 0.8741 & 0.8692 & 0.8841 & 0.8638 & 0.8682 \\ 
$3, 30, 200$ & 0.0001 & 0.0001 & 0.0001 & 0.0000 & -0.0000 & 0.0002 & 
$3, 30, 200$ & 0.4747 & 0.4389 & 0.4469 & 0.4851 & 0.4376 & 0.4356 \\ 
$3, 30, 200$ & 0.2554 & 0.3195 & 0.3296 & 0.3090 & 0.3368 & 0.3278 \\ 
$3, 30, 200$ & 0.2193 & 0.1194 & 0.1173 & 0.1761 & 0.1009 & 0.1077 & 
$9, 30, 200$ & 0.9310 & 0.9260 & 0.9256 & 0.9289 & 0.9275 & 0.9284 \\ 
$9, 30, 200$ & 0.9310 & 0.9259 & 0.9258 & 0.9287 & 0.9272 & 0.9285 \\ 
$9, 30, 200$ & 0.0000 & 0.0002 & -0.0002 & 0.0002 & 0.0002 & -0.0002 & 
$9, 30, 200$ & 0.1828 & 0.1964 & 0.2022 & 0.1892 & 0.1947 & 0.1948 \\ 
$9, 30, 200$ & 0.1817 & 0.2045 & 0.2127 & 0.1961 & 0.2039 & 0.2046 \\ 
$9, 30, 200$ & 0.0011 & -0.0082 & -0.0105 & -0.0069 & -0.0093 & -0.0098 & 
$27, 30, 200$ & 0.9429 & 0.9418 & 0.9418 & 0.9421 & 0.9412 & 0.9409 \\ 
$27, 30, 200$ & 0.9430 & 0.9419 & 0.9418 & 0.9421 & 0.9413 & 0.9407 \\ 


SystemExit: 

In [ ]:
generate_plot(x_label_all, single_smooth_post_L_all, single_smooth_post_H_all, 'Posterior Predictive', 'Small Smooth', 'single_smooth_post_all')
generate_plot(x_label_all, single_score_post_L_all, single_score_post_H_all, 'Posterior Predictive', 'PMI', 'single_score_post_all')
generate_plot(x_label_all, single_post_base_loss_L_all, single_post_base_loss_H_all, 'Cross Entropy', 'Average Cross Entropy', 'single_post_base_loss_all')
generate_plot(x_label_all, single_post_base_acc_L_all, single_post_base_acc_H_all, 'Accuracy', 'Average Accuracy', 'single_post_base_acc_all')
generate_plot(x_label_all, smooth_post_L_all, smooth_post_H_all, 'Posterior Predictive', 'Small Smooth', 'smooth_post_all')
generate_plot(x_label_all, score_post_L_all, score_post_H_all, 'Posterior Predictive', 'PMI', 'score_post_all')
generate_plot(x_label_all, post_base_loss_L_all, post_base_loss_H_all, 'Cross Entropy', 'Average Cross Entropy', 'post_base_loss_all')
generate_plot(x_label_all, post_base_acc_L_all, post_base_acc_H_all, 'Accuracy', 'Average Accuracy', 'post_base_acc_all')

In [ ]:
# print(acc_all_all)
# print(single_total_summary[2].total_result.score_loss)
# print(single_total_summary[2].total_result.score_acc)
# print(single_total_summary[2].total_result.err_loss)
# print(single_total_summary[2].total_result.err_acc)
# print(single_total_summary[2].total_result.post_loss)
# print(single_total_summary[2].total_result.post_acc)
# print(single_total_summary[2].total_result.acc_loss)
# print(single_total_summary[2].total_result.acc_acc)
# print(single_total_summary[2].total_result.smooth_loss)
# print(single_total_summary[2].total_result.smooth_acc)

In [ ]:
# profiler.stop()

# profiler.print()